# Alpaca-LoRA SageMaker Inference

This is a sample code to deploy Alpaca-LoRA to SageMaker Endpoint.

In [ ]:
!pip install "sagemaker>=2.143.0" -U

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFaceModel

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## Package and Upload Model

In [ ]:
%cd scripts
!tar -czvf ../package.tar.gz *
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"Alpaca")
model_path

## Deploy Model

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.serializers import JSONSerializer

endpoint_name = "Alpaca-Inference"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="1.13",
    py_version='py39',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "base_model": "huggyllama/llama-7b",
            "lora_weights": "tloen/alpaca-lora-7b",
            "peft": True,
            "load_8bit": True,
            "prompt_template": "alpaca",
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "3600"
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    # async_inference_config=AsyncInferenceConfig()
)

## Run Inference

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor_client = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)
# predictor_client = AsyncPredictor(
#     predictor=predictor_client,
#     name=endpoint_name
# )
data = {
    "instruction": "Answer the question below",
    "input": "What is AWS"
}
response = predictor_client.predict(
    data=data
)
print(response)

In [ ]:
# With Boto3

import boto3
import json

endpoint_name = "Alpaca-Inference"
sagemaker_client = boto3.client('sagemaker-runtime')

data = {
    "instruction": "Answer the question below",
    "input": "What is AWS",
    "max_new_tokens": 128,
    "temperature": 0.3,
}

response = sagemaker_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(data)
)

result = json.loads(response['Body'].read())
print(result)

## Benchmark Speed

12.4 s ± 101 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%timeit response = predictor_client.predict(data=data)

## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()